# LangChain Application: YouTube Summary

## Import

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import tiktoken
import os


## Environment Variables and OPENAI API
- Save personal API key in `.env` file and use `load_dotenv()` to load environment variables.
- Specify `OPENAI_API_KEY` in the self-defined `.env`.

In [2]:
load_dotenv('/Users/alvinchen/.env')

True

## Loading YouTube

- LangChain offers many loaders and transforms various inputs into standardized Document format.

:::{note}
Check the official documentation of [Document Loaders](https://python.langchain.com/v0.2/docs/integrations/document_loaders/) for more detail.
:::

In [3]:
## Specify a youtube link
loader = YoutubeLoader.from_youtube_url('https://www.youtube.com/watch?v=_BMAkffp2CA')
doc = loader.load()

In [4]:
print(type(doc[0]))

<class 'langchain_core.documents.base.Document'>


In [5]:
print(doc[0].metadata)

{'source': '_BMAkffp2CA'}


In [6]:
transcript= doc[0].page_content

print(transcript)

now we've been reporting in the last half hour those new lines coming out from the White House Israel agreeing to 4H hour military pauses in Northern Gaza also information about two humanitarian corridors to allow people to flee the hostilities so let's head to Washington let's talk to our state department correspondent Barbara PL Usher and Barbara tell us more then in terms of what the Americans have been saying in the last little while what they have said is that they've reached agreement with the Israelis to have a 4-Hour pause every day um in order to allow Palestinians in Northern Gaza which is where the main military operation is to flee safely to the South and they Israel we have been told would give a three-hour notice every day when that pause would be uh and the Palestinians if they wanted to leave would be taking uh safe passage corridors which reportedly have actually been um operating for several days now but now they would have um a cessation of military operations um for

## Counting Tokens

In [7]:

def count_tokens(string: str, model: str = "gpt-4") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [8]:
#model="gpt-3.5-turbo"
model="gpt-4" ## if you cannot use `gpt-4`, use `gpt-3.5-turbo` then.

count_tokens(transcript, model)

744

## Summarizing YouTube

- Now we can make use of LangChain to generate the summary of the video content.
- Three steps:
    - Create a prompt template for summary
    - Add the current transcript to the template
    - Pass it to the LLM for summarization

In [9]:
llm = ChatOpenAI(model=model, temperature=0.6)


In [10]:
## Prompt Template

SUMMARY_PROMPT = """Summarize the video with the transcript delimited by triple backticks in a concise and natural way. \
Present the summaries in the form of 5 key takeaways. \
Transcript: ```{transcript}```"""

SUMMARY_PROMPT2 = """Summarize the video with the transcript delimited by triple backticks in Mandarin Chinese. \
    Present the summaries in the form of 5 key takeways. \
        Transcript: ```{transcript}```"""

In [11]:


def summarize_video(transcript, template=SUMMARY_PROMPT, model="gpt-3.5-turbo"):
    
    prompt = ChatPromptTemplate.from_template(template)
    # passing the transcript to the template
    formatted_prompt = prompt.format_messages(transcript=transcript)

    # initialize model
    llm = ChatOpenAI(model=model, temperature=0.1)
    # generate summary
    summary = llm.invoke(formatted_prompt)
    return summary

In [12]:
summary = summarize_video(transcript=transcript)
print(summary.content)

Key Takeaways:
1. The White House and Israel have agreed to 4-hour military pauses in Northern Gaza to allow Palestinians to flee safely to the South.
2. The pauses are also intended to facilitate the delivery of aid into Gaza and potentially help in the release of hostages.
3. President Biden had asked for a longer pause than 3 days, showing frustration with the delay in reaching an agreement.
4. The pressure from Washington on Israel's Prime Minister to agree to longer breaks in the fighting has been increasing as the humanitarian crisis in Gaza worsens.
5. The focus is on getting aid into Gaza, addressing the dire situation, and potentially securing the release of hostages through diplomatic efforts.


In [13]:
summary2 = summarize_video(transcript=transcript, template=SUMMARY_PROMPT2)
print(summary2.content)

关于白宫同意以色列在北加沙进行4小时的军事停火以及开放两个人道主义通道的新消息，美国与以色列达成协议，每天进行4小时的停火，以便让北加沙的巴勒斯坦人安全地向南逃离。白宫希望利用这些停火来向加沙运送更多援助物资，最终可能还会利用停火来促进人质的释放。

5个要点：
1. 美国与以色列达成协议，每天进行4小时的军事停火，以便让北加沙的巴勒斯坦人安全地向南逃离。
2. 白宫希望利用停火来向加沙运送更多援助物资，每天希望能够送达150辆卡车的援助物资。
3. 总统拜登曾要求以色列总理进行为期三天的停火，以便释放人质和运送更多援助物资。
4. 美国一直在向以色列施加压力，希望获得更长时间的停火，以解决加沙的人道危机和释放人质。
5. 白宫和国务院一直在努力争取停火，以便向加沙运送援助物资，并帮助巴勒斯坦平民前往安全地区。


## Chat with the YouTube Content

In [14]:
## Pretrained embedding models
embeddings = OpenAIEmbeddings()


In [28]:

def create_db_from_youtube_video_url(video_url: str) -> FAISS:
    ## document loader
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()
    
    ## text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)
    
    ## vector store
    db = FAISS.from_documents(docs, embeddings)
    return db


def get_response_from_query(db, query, k=4):
    """
    gpt-3.5-turbo can handle up to 4097 tokens. Setting the chunksize to 1000 and k to 4 maximizes
    the number of tokens to analyze.
    """

    ## vector store simlarity check
    docs = db.similarity_search(query, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])

    ## LLM 
    llm = ChatOpenAI(model_name="gpt-3.5-turbo")

    ## Prompt
    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        You are a helpful assistant that that can answer questions about youtube videos 
        based on the video's transcript.
        
        Answer the following question: {question}
        By searching the following video transcript: {docs}
        
        Only use the factual information from the transcript to answer the question.
        
        If you feel like you don't have enough information to answer the question, say "I don't know".
        
        Your answers should be verbose and detailed.
        """,
    )

    ## Chain
    # chain = LLMChain(llm=llm, prompt=prompt)
    chain = prompt | llm
    ## Invoke
    # response = chain.invoke(question=query, docs=docs_page_content)
    response = chain.invoke({"question": query, "docs": docs_page_content})

    return response.content, docs

In [29]:
db = create_db_from_youtube_video_url(video_url= 'https://www.youtube.com/watch?v=SFnMTHhKdkw')

In [30]:
get_response_from_query(db=db, query="Summarize the talk in 50 words.")

("The talk discusses the speaker's lifelong involvement in education, highlighting the importance of human connection in learning. They emphasize the impact educators can have on students' lives by building relationships and showing care and support. Personal anecdotes about the speaker's mother's dedication to her students further illustrate this point. Through these stories, the speaker emphasizes the transformative power of education and the role educators play in shaping students' lives.",
 [Document(page_content="Transcriber: Joseph Geni\nReviewer: Morton Bast I have spent my entire life either at the schoolhouse,\non the way to the schoolhouse, or talking about what happens\nin the schoolhouse. (Laughter) Both my parents were educators, my maternal grandparents were educators, and for the past 40 years,\nI've done the same thing. And so, needless to say, over those years I've had a chance\nto look at education reform from a lot of perspectives. Some of those reforms have been goo

In [31]:
get_response_from_query(db=db, query = "How much does the speaker like teaching?", k = 1)

("Based on the transcript provided, it is clear that the speaker values building relationships with students and believes that teaching goes beyond just delivering a lesson. The speaker emphasizes the importance of liking the kids they teach, as they mention that kids don't learn from people they don't like. This indicates that the speaker values the connection and relationship they have with their students. \n\nFurthermore, the speaker mentions that they taught a lesson on ratios, showing that they are actively engaged in the teaching process and are willing to put effort into creating engaging and effective lessons for their students. This demonstrates a genuine passion for teaching and a desire to see their students succeed.\n\nOverall, based on the transcript, it can be inferred that the speaker highly values teaching and the relationships they build with their students. They believe that liking the kids they teach is crucial for effective learning to take place.",
 [Document(page_

In [32]:
get_response_from_query(db=db, query = "Identify examples of teaching given by the speaker.", k = 5)

("The speaker in the video provides several examples of teaching throughout the transcript. Firstly, the speaker mentions watching people teach for years, observing both the best and the worst methods. They also talk about the importance of building relationships with students, mentioning the significance of seeking to understand them and apologizing when necessary. The speaker recounts teaching a lesson on ratios, despite not being confident in their math abilities, demonstrating their dedication to educating their students. Additionally, the speaker describes a moment when a student performed poorly on a quiz, but instead of focusing on the negative grade, the speaker highlighted the two questions the student answered correctly, showing encouragement and positivity. The speaker also reflects on a difficult class they had, expressing their emotional struggle to help the students improve academically and raise their self-esteem simultaneously. Furthermore, the speaker shares a story ab

## An Integrated YouTube Summarizer

- Context window of the models:
    - `gpt-3.5-turbo`: 4,096 tokens
    - `gpt-4`: 8,192 tokens
    - `gpt-4-32k`: 32,768 tokens
    - `gpt-4-turbo`: 128,000 tokens

In [33]:
def get_transcript(url: str) -> str:
    """
    Returns the transcript and title from a YouTube URL.

    Parameters:
    url (str): The YouTube URL from which the transcript and title will be extracted.

    Returns:
    transcript (str): The transcript of the video.
    """
    try:
        loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
        docs = loader.load()
        if docs:
            doc = docs[0]
            transcript = doc.page_content
            print(transcript)
            return transcript
        else:
            return None
    except Exception as e:
        print(f"Failed to load transcript and title from URL {url}: {e}")
        return None

In [34]:
TEMPLATE_PROMPT = """Summarize the video with the transcript delimited by triple backticks. \n
Answer the questions delimited by single backticks. If no questions provided, just create a general summary. \n
Questions: ` {questions} ` \n
Transcript: ```{transcript}```"""

In [44]:
def summarize_with_questions(transcript, questions, model):
    prompt = ChatPromptTemplate.from_template(TEMPLATE_PROMPT)
    formatted_prompt = prompt.format_messages(transcript=transcript, questions=questions)
    print("Formatted prompt: ", formatted_prompt)
    llm = ChatOpenAI(model=model, temperature=0.6)
    summary = llm.invoke(formatted_prompt)
    return summary

In [45]:
def summarize_from_url(url, questions):
    transcript = get_transcript(url)
    token_count = count_tokens(transcript)
    # select model
    if token_count < 6000:
        model = "gpt-4"
    elif token_count < 30000:
        model = "gpt-4-32k"
    else:
        return f"Summary unavailable for transcripts over 30k tokens. Your transcript has {token_count} tokens."
    
    summary = summarize_with_questions(transcript, questions, model)
    return summary

In [42]:
# example videos
urls = [
    "https://www.youtube.com/watch?v=_BMAkffp2CA"
]

In [46]:
url = urls[0]
questions = "What is the main idea of the news coverage?"

summary = summarize_from_url(url, questions)


now we've been reporting in the last half hour those new lines coming out from the White House Israel agreeing to 4H hour military pauses in Northern Gaza also information about two humanitarian corridors to allow people to flee the hostilities so let's head to Washington let's talk to our state department correspondent Barbara PL Usher and Barbara tell us more then in terms of what the Americans have been saying in the last little while what they have said is that they've reached agreement with the Israelis to have a 4-Hour pause every day um in order to allow Palestinians in Northern Gaza which is where the main military operation is to flee safely to the South and they Israel we have been told would give a three-hour notice every day when that pause would be uh and the Palestinians if they wanted to leave would be taking uh safe passage corridors which reportedly have actually been um operating for several days now but now they would have um a cessation of military operations um for

In [47]:

print(summary)


content="The main idea of the news coverage is the agreement reached by Israel to implement a daily four-hour pause in military operations in Northern Gaza. This agreement, brokered by the United States, is intended to provide safe passage for Palestinians to flee the hostilities. Israel will give a three-hour notice each day when this pause will occur. The U.S. hopes to use these pauses to deliver more aid to Gaza, aiming to send up to 150 trucks daily, and possibly facilitate the release of hostages. The news coverage also mentions that President Biden had previously asked the Israeli Prime Minister for a longer pause, and he expressed frustration that the situation has taken longer than expected. Despite supporting Israel's stance that a ceasefire would benefit Hamas, the U.S. is pressing for a break in the fighting to address the dire humanitarian crisis in Gaza."


## Resources

- [OPENAI API Models](https://platform.openai.com/docs/models): For more detail information about the models supported by OPENAI.